In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [4]:
stack = 'MD594'
#yaml_file = 'shape_params-aws.yaml'
yaml_file = 'shape_params.yaml'
img_file = '/CSHL_hsv/'+stack+'/'
scripts_dir = os.environ['REPO_DIR']

In [39]:
Section.drop()

`kui_diffusionmap`.`section` (153 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [5]:
@schema
class Section(dj.Manual):
    definition = """
    section_id  : int   # section number
    """

In [6]:
import pickle
fname = os.path.join('CSHL_data_processed', stack, 'All_patch_locations.pkl')
all_patch_locations = pickle.load(open(os.environ['ROOT_DIR']+fname, 'rb'), encoding='latin1')

In [7]:
valid_sections = []
for section in all_patch_locations.keys():
    if all_patch_locations[section]:
        valid_sections.append(section)

In [11]:
from glob import glob
fp = os.environ['ROOT_DIR']+'CSHL_scoremaps_new/down32/'+stack+'/'
fnames = [dir for dir in glob(fp+'SC/*')]
exist = []
for item in fnames:
    try:
        point = item.rfind('SC_')
        point+= 3
        end = item.rfind('.')
        if end-point>5:
            end = item.rfind('_')
        exist.append(int(item[point:end]))
    except:
        continue

In [13]:
fnames = [dir for dir in glob(fp+'DC/*')]
exist2 = []
for item in fnames:
    try:
        point = item.rfind('DC_')
        point+= 3
        end = item.rfind('.')
        if end-point>5:
            end = item.rfind('_')
        exist2.append(int(item[point:end]))
    except:
        continue

In [18]:
ig = []
for sec in exist:
    if sec in exist2:
        ig.append(sec)

In [19]:
ig.extend([160,175,179])

In [21]:
com_sections = []
for sec in valid_sections:
    if sec not in ig:
        com_sections.append(sec)

In [22]:
len(ig), len(com_sections)

(115, 149)

In [38]:
len(valid_sections)

264

In [23]:
for section in com_sections:
    print("\nAdding "+str(section)+' to the database')
    Section.insert1(dict(section_id=section)
                      ,skip_duplicates=True)


Adding 164 to the database

Adding 170 to the database

Adding 176 to the database

Adding 178 to the database

Adding 180 to the database

Adding 181 to the database

Adding 182 to the database

Adding 183 to the database

Adding 184 to the database

Adding 185 to the database

Adding 186 to the database

Adding 188 to the database

Adding 189 to the database

Adding 190 to the database

Adding 191 to the database

Adding 192 to the database

Adding 194 to the database

Adding 195 to the database

Adding 196 to the database

Adding 197 to the database

Adding 198 to the database

Adding 199 to the database

Adding 201 to the database

Adding 202 to the database

Adding 203 to the database

Adding 204 to the database

Adding 205 to the database

Adding 206 to the database

Adding 207 to the database

Adding 208 to the database

Adding 209 to the database

Adding 210 to the database

Adding 211 to the database

Adding 212 to the database

Adding 213 to the database

Adding 214 to the d

In [11]:
Section.fetch(as_dict=True)

[OrderedDict([('section_id', 155)]),
 OrderedDict([('section_id', 160)]),
 OrderedDict([('section_id', 164)]),
 OrderedDict([('section_id', 170)]),
 OrderedDict([('section_id', 175)]),
 OrderedDict([('section_id', 176)]),
 OrderedDict([('section_id', 178)]),
 OrderedDict([('section_id', 179)]),
 OrderedDict([('section_id', 180)]),
 OrderedDict([('section_id', 181)]),
 OrderedDict([('section_id', 182)]),
 OrderedDict([('section_id', 183)]),
 OrderedDict([('section_id', 184)]),
 OrderedDict([('section_id', 185)]),
 OrderedDict([('section_id', 186)]),
 OrderedDict([('section_id', 188)]),
 OrderedDict([('section_id', 189)]),
 OrderedDict([('section_id', 190)]),
 OrderedDict([('section_id', 191)]),
 OrderedDict([('section_id', 192)]),
 OrderedDict([('section_id', 194)]),
 OrderedDict([('section_id', 195)]),
 OrderedDict([('section_id', 196)]),
 OrderedDict([('section_id', 197)]),
 OrderedDict([('section_id', 198)]),
 OrderedDict([('section_id', 199)]),
 OrderedDict([('section_id', 201)]),
 

In [11]:
%%time
! python ../Scoremap_V2.py 'MD594' 200 'shape_params.yaml'

/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
/Users/kuiqian/Github/shapeology_code/scripts/lib/utils.py:22: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.D=yaml.load(open(yamlFile,'r'))
ALREADY DOWNLOADED FILE
run cmd= aws s3 cp s3://mousebrainatlas-data/CSHL_grid_features/MD594/200.pkl /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_grid_features/MD594/200.pkl
fatal error: An error occurred (404) when calling the HeadObject operation: Key "CSHL_grid_features/MD594/200.pkl" does not exist
0 15400
1000 15400
2000 15400
3000 15400
4000 15400
5000 15400
6000 15400
700

In [17]:
feature_vector = np.fromfile(open(os.environ['ROOT_DIR']+'CSHL_grid_features/MD594/200/200_224_1344.npz', 'br'))

In [6]:
valid = ['kui@172.31.11.31', 'kui@172.31.13.156', 'kui@172.31.0.238', 'kui@172.31.15.155', 
         'kui@172.31.4.94', 'kui@172.31.9.31', 'kui@172.31.8.90', 'kui@172.31.13.143']

In [7]:
ter = []
for pid in list(pd.DataFrame(schema.jobs)['user']):
    if pid not in valid:
        ter.append(pid)

In [8]:
len(ter), len(list(pd.DataFrame(schema.jobs)['user']))

(41, 49)

In [20]:
pid = ter[0]
'user="'+pid+'"'
(schema.jobs & 'user="'+pid+'"' )

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__score_map,013d407166ec4fa56eb1e1f8cbe183b9,reserved,=BLOB=,,=BLOB=,kui@172.31.24.217,ip-172-31-24-217,8390,31102,2019-08-16 02:09:08


In [22]:
for pid in ter:
    (schema.jobs & 'user="'+pid+'"' ).delete()

In [13]:
(schema.jobs & 'status="reserved"' ).delete()
(schema.jobs & 'status="error"' ).delete()

In [4]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [9]:
(schema.jobs & 'status="error"' )

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__score_map,9b04d152845ec0a378394003c96da594,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,kui@172.31.14.23,ip-172-31-14-23,8442,31445,2019-08-17 00:29:44
__score_map,e4a6222cdb5b34375400904f03d8e6a5,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,kui@172.31.13.30,ip-172-31-13-30,8398,31455,2019-08-16 23:35:11


In [7]:
(schema.jobs & 'status="reserved"' )

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [6]:
ScoreMap.drop()

`kui_diffusionmap`.`__score_map` (149 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [5]:
table = ScoreMap.fetch(as_dict=True)
df = pd.DataFrame(table)
df

,section_id,structure_number
0,164,28
1,170,28
2,176,28
3,178,28
4,180,28
5,181,28
6,182,28
7,183,28
8,184,28
9,185,28


In [47]:
! aws s3 ls mousebrainatlas-data/CSHL_scoremaps_new/MD594/10N/

2019-08-15 18:08:03    3704062 10N_100.tif
2019-08-15 18:08:13    3684078 10N_101.tif
2019-08-15 18:08:18    3607566 10N_102.tif
2019-08-15 22:05:43    3693022 10N_103.tif
2019-08-15 22:06:00    3745022 10N_104.tif
2019-08-15 22:43:09    3521486 10N_105.tif
2019-08-15 22:51:09    3732398 10N_106.tif
2019-08-15 18:56:48    3805950 10N_107.tif
2019-08-15 18:56:47    3716414 10N_108.tif
2019-08-15 18:57:27    3770782 10N_109.tif
2019-08-15 18:58:30    3770702 10N_110.tif
2019-08-15 18:58:04    3819694 10N_111.tif
2019-08-15 18:58:34    3811198 10N_112.tif
2019-08-15 18:58:42    3779486 10N_113.tif
2019-08-15 22:54:01    3823742 10N_114.tif
2019-08-15 18:57:51    3785438 10N_115.tif
2019-08-15 22:53:08    3846718 10N_116.tif
2019-08-15 22:52:38    3834158 10N_117.tif
2019-08-15 19:01:01    3874270 10N_118.tif
2019-08-15 22:54:45    3808990 10N_119.tif
2019-08-15 18:57:39    3807950 10N_120.tif
2019-08-15 18:59:45    3827086 10N_121.tif
2019-08-15 18:57:00    3687998 10N_122.tif
2019-08-15 

In [42]:
ImageMap.drop()

`kui_diffusionmap`.`__image_map` (0 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [12]:
! python HSV_datajoint_v2.py 'local' 'MD594'

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
Connecting kui@ucsd-demo-db.datajoint.io:3306
ALREADY DOWNLOADED FILE
populating for  94
populating for  95
populating for  96
populating for  97
populating for  98
populating for  99
populating for  100
populating for  101
populating for  102
populating for  103
populating for  104
populating for  105
populating for  106
populating for  107
populating for  108
populating for  109
populating for  110
populating for  111
populating for  112
populating for  113
populating for  114
populating for  115
populating for  116
populating for  117
populating for  118
populating for  119
populating for  120
populating for  121
populating for  122
populating for  123
populating for  124
populating for  125
populating for  126
populating for  127
populating

In [57]:
! python ../HSV_v2.py 'MD589' 118 'shape_params.yaml'

/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
/Users/kuiqian/Github/shapeology_code/scripts/lib/utils.py:22: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.D=yaml.load(open(yamlFile,'r'))
run cmd= aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_prep2_lossless_gray/Placeholder_prep2_lossless_gray.tif /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD589/MD589_prep2_lossless_gray/Placeholder_prep2_lossless_gray.tif
fatal error: An error occurred (404) when calling the HeadObject operation: Key "CSHL_data_proces

In [58]:
stack = 'MD589'
fp = os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt')
with open(os.environ['ROOT_DIR']+fp, 'r') as f:
    fn_idx_tuples = [line.strip().split() for line in f.readlines()]
    section_to_filename = {int(idx): fn for fn, idx in fn_idx_tuples}

In [61]:
section_to_filename[117]

'MD589-IHC20-2015.07.30-22.36.03_MD589_3_0060'

In [120]:
schema.connection.close()

In [59]:
Sqlite.fetch(as_dict=True)

[OrderedDict([('section_id', 94),
              ('size_of_db', 13676),
              ('size_of_pkl', 3669590)]),
 OrderedDict([('section_id', 95),
              ('size_of_db', 12840),
              ('size_of_pkl', 3776965)]),
 OrderedDict([('section_id', 96),
              ('size_of_db', 13336),
              ('size_of_pkl', 3554139)]),
 OrderedDict([('section_id', 97),
              ('size_of_db', 13340),
              ('size_of_pkl', 3902568)]),
 OrderedDict([('section_id', 98),
              ('size_of_db', 13996),
              ('size_of_pkl', 3775501)]),
 OrderedDict([('section_id', 99),
              ('size_of_db', 13152),
              ('size_of_pkl', 3763688)]),
 OrderedDict([('section_id', 100),
              ('size_of_db', 13844),
              ('size_of_pkl', 3985848)]),
 OrderedDict([('section_id', 101),
              ('size_of_db', 13524),
              ('size_of_pkl', 3814819)]),
 OrderedDict([('section_id', 102),
              ('size_of_db', 13963),
              ('size_o

In [138]:
schema.size_on_disk

16384